In [1]:
import pandas as pd
import cx_Oracle
import datetime

C:\Users\hkccloud01\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\hkccloud01\anaconda3\lib\site-packages\numpy\.libs\libopenblas.el2c6ple4zyw3eceviv3oxxgrn2nrfm2.gfortran-win_amd64.dll
C:\Users\hkccloud01\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\hkccloud01\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### db

In [2]:
def get_conn_ora():
    
    username = "DPKMAPP"
    password = "Kms12#$"
    host = "61.106.19.217"
    port = 33389
    database = "CDB1"
    dsn = f'{host}:{port}/{database}'
    encoding = 'UTF-8'

    # 오라클 클라이언트를 선언: windows 환경변수에 path를 잡으면 필요없음
    try:
        cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle\instantclient_21_12")
    except Exception as e:
        pass

    try:
        connection = cx_Oracle.connect(username, password, dsn, encoding=encoding)
    except Exception as ex:
        print('Could not connect to database:', ex)
        return ex

    print("SUCCESS: Connecting database succeeded")
    return connection

def get_kms_datadf(sql):
    try:
        connection = get_conn_ora()
     
        with connection.cursor() as cursor:
            cursor.execute(sql)
            rows = cursor.fetchall()
            cols = [x[0].lower() for x in cursor.description]
   
        df = pd.DataFrame(rows, columns = cols)
        for c in df.columns:
            if df[c].dtype == object:
                df[c] = df[c].astype("string")

        connection.close()

    except Exception as ex:
        err, = ex.args
        print('error code :', err.code)
        print('error message :', err.message)
        return None
   
    if len(df) ==0:
        print('No data found')  
        return None
    return df


In [ ]:
#컨텐츠 리스트
ctgr_cd ='CA'
temp = f" AND CTGR_ID =   '{ctgr_cd}'" if ctgr_cd !='CA' else  " AND CTGR_ID = 'CA' "
sql = f"""
        SELECT
            AA.CNTNT_ID        
            , AA.TITL 
            , (SELECT CD_NM FROM TBCTKC10 WHERE CD = AA.SAVE_STAT_CD AND GROUP_CD = 'SAVE_STAT_CD') AS CD_NM
            , AA.CTGR_ID
            , AA.CTGR_PATH
            , AA.REG_DTTM
            , AA.UPD_DTTM
            , ((SELECT COUNT(FILE_KEY) FROM TBCTKK08 WHERE CNTNT_ID = AA.CNTNT_ID AND USE_YN='Y') +
               (SELECT COUNT(FILE_KEY) FROM TBCTKK22 WHERE CNTNT_ID = AA.CNTNT_ID AND USE_YN='Y')) AS ATT_CNT
            , (SELECT count(*) FROM TBCGPT10 G WHERE G.CNTNT_ID = AA.CNTNT_ID ) AS EMB_CNT
            FROM (
            SELECT A.CNTNT_ID
                ,(CASE WHEN A.SAVE_STAT_CD = '00' THEN 
                            (SELECT 
                            (CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END) AS AUTH_YN
                            FROM TBCTKC21 A
                            INNER JOIN TBCTKC23 B
                                ON  A.ATRT_GROUP_ID = B.ATRT_GROUP_ID
                            WHERE A.ETC_INFO01 > '50') 
                    ELSE 'Y' END) AS VIEW_YN
                , A.TITL
                , A.CTGR_ID
                , A.REG_DTTM
                , A.UPD_DTTM
                , BB.CTGR_PATH
                , A.SAVE_STAT_CD
            FROM TBCTKK01 A
            INNER JOIN
                (  SELECT  CTGR_ID
                    , CTGR_NM
                    , CTGR_PATH
                    , CONNECT_BY_ISLEAF AS IS_LEAF
                    FROM TBCTKK12
                    START WITH USE_YN ='Y'
                {temp}
                CONNECT BY PRIOR CTGR_ID = HGRK_CTGR_ID AND USE_YN ='Y' ) BB
            ON  A.CTGR_ID = BB.CTGR_ID
            ) AA
        WHERE AA.VIEW_YN='Y'
        ORDER BY AA.CNTNT_ID desc
    """
df = get_kms_datadf(sql)
df['reg_dt'] = df['reg_dttm'].dt.date
df['site']   = df['ctgr_path'].apply(lambda x: x.split('>')[1])
df_a = df[df['cd_nm']!='삭제'].groupby(['reg_dt'])[['cntnt_id']].count().rename(columns={'cntnt_id':'reg_cnt'}).sort_index(ascending=False)

In [ ]:
from_dt = '20201219'
to_dt = '20240119'
#일자별 등록현황
sql = f"""
   SELECT  	ROW_NUMBER() OVER (ORDER BY A.INQ_CNT DESC)  AS ROW_NUMBER
    , A.CNTNT_ID
    , A.DT
	, A.TITL
	, A.INQ_CNT
	, A.CTGR_PATH
	, A.REG_DTTM
	, DPKMAPP.FNCTK_GET_USERNAME(A.REGR_ID, A.REGR_DEPT_CD, 'DEPT') AS REGR_NM
FROM (
	SELECT   
			 A.CNTNT_ID
		   , A.DT
		   , MAX(B.TITL)							AS TITL
		   , SUM(nvl(A.INQ_CNT, 0))					AS INQ_CNT
		   , MAX(B.CTGR_ID)							AS CTGR_ID
		   , MAX(C.CTGR_PATH) 						AS CTGR_PATH
		   , MAX(B.REG_DTTM) 						AS REG_DTTM
    	   , MAX(B.REGR_ID)							AS REGR_ID
    	   , MAX(B.REGR_DEPT_CD)					AS REGR_DEPT_CD
	  FROM DPKMAPP.TBCTKK03 A	   /* 컨텐츠 마스터  */
INNER JOIN DPKMAPP.TBCTKK01 B ON (A.CNTNT_ID = B.CNTNT_ID)							
  INNER JOIN (
            SELECT CTGR_ID
                 , CTGR_NM
                 , CTGR_PATH
            FROM DPKMAPP.TBCTKK12
            START WITH 1=1
			 AND CTGR_ID      ='CA'
            CONNECT BY PRIOR CTGR_ID = HGRK_CTGR_ID AND USE_YN ='Y'
       ) C ON (B.CTGR_ID = C.CTGR_ID)  /* 카테고리 */
	 WHERE 1=1
 	AND DT BETWEEN {from_dt} AND {to_dt}							
  GROUP BY A.CNTNT_ID, A.DT
     ) A
"""
df_inq = get_kms_datadf(sql)
df_inq['req_dt'] = df_inq['reg_dttm'].dt.date
df_inq = df_inq.groupby(['req_dt'])[['inq_cnt']].sum().sort_index(ascending=False)

### loader

####  unstructed

In [ ]:
from unstructured.partition.pdf import partition_pdf
path = "static/"
''' 라이브러리문제로 실행 오류발생'''
# Extract images, tables, and chunk text
raw_pdf_elements = partition_pdf(
    filename= path +"test.pdf",
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=path,
)

### PDFloader

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyMuPDFLoader

In [ ]:
file_name = 'static/test.pdf'
loader = PyPDFLoader(file_name)
documents = loader.load_and_split()

In [ ]:
# doc = fitz.open("static/test.pdf")
doc = fitz.open("static/chinese-table.pdf")

In [ ]:
# doc.get_toc()
doc.metadata
# doc.page_count
# doc.load_page(0)

In [ ]:
def show_image(item, title=""):
    """Display a pixmap.

    Just to display Pixmap image of "item" - ignore the man behind the curtain.

    Args:
        item: any PyMuPDF object having a "get_pixmap" method.
        title: a string to be used as image title

    Generates an RGB Pixmap from item using a constant DPI and using matplotlib
    to show it inline of the notebook.
    """
    DPI = 150  # use this resolution
    import numpy as np
    import matplotlib.pyplot as plt

    # %matplotlib inline
    pix = item.get_pixmap(dpi=DPI)
    img = np.ndarray([pix.h, pix.w, 3], dtype=np.uint8, buffer=pix.samples_mv)
    plt.figure(dpi=DPI)  # set the figure's DPI
    plt.title(title)  # set title of image
    _ = plt.imshow(img, extent=(0, pix.w * 72 / DPI, pix.h * 72 / DPI, 0))
    
import fitz
if not hasattr(fitz.Page, "find_tables"):
    raise RuntimeError("This PyMuPDF version does not support the table feature")
    
'''표 인식이 잘 안됨'''
# doc = fitz.open("static/test.pdf")
doc = fitz.open("static/chinese-table.pdf")
print(doc)
header = "Header"  # text in header
footer = "Page %i of %i"  # text in footer
for page in doc:
#     print(dir(page))
#     page.insert_text((50, 50), header)  # insert header
#     page.insert_text(  # insert footer 50 points above page bottom
#         (50, page.rect.height - 50),
#         footer % (page.number + 1, doc.page_count),
#     )
    print(page.get_text())


###### find table는 잘 인식되는듯하나 병합셀이 있으면 인식 불능으로 생각됨

In [ ]:
''' find tables '''
page = doc[0]
tabs = page.find_tables()  # detect the tables
for i,tab in enumerate(tabs):  # iterate over all tables
    for cell in tab.header.cells:
        page.draw_rect(cell,color=fitz.pdfcolor["red"],width=0.3)
    page.draw_rect(tab.bbox,color=fitz.pdfcolor["green"])
    print(f"Table {i} column names: {tab.header.names}, external: {tab.header.external}")
show_image(page, f"Table & Header BBoxes")

In [ ]:
page = doc[0]
tabs = page.find_tables(horizontal_strategy="text")
# print(len(tabs.tables))
print( f"{len(tabs.tables)} tables on {page}" )
tab = tabs[0]
for line in tab.extract():  # print cell text for each row
    print(line)
df = tab.to_pandas()

In [ ]:
# choose the second table for conversion to a DataFrame
tab = tabs[0]
df2 = tab.to_pandas()

### html header splitter

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import HTMLHeaderTextSplitter
from langchain_community.document_loaders import BSHTMLLoader

url = "docs/여비규정.html"

headers_to_split_on = [  # 분할할 HTML 헤더 태그와 해당 헤더의 이름을 지정합니다.
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]

# HTML 헤더를 기준으로 텍스트를 분할하는 HTMLHeaderTextSplitter 객체를 생성합니다.
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# URL에서 텍스트를 가져와 HTML 헤더를 기준으로 분할합니다. 
#(split_text_from_url(),split_text_from_file(),split_text()
# html_header_splits = html_splitter.split_text_from_url(url)
# loader = BSHTMLLoader(url,'utf-8')
# documents = loader.load_and_split()
html_header_splits = html_splitter.split_text_from_file(url)

chunk_size = 900  # 텍스트를 분할할 청크의 크기를 지정합니다.
chunk_overlap = 100  # 분할된 청크 간의 중복되는 문자 수를 지정합니다.
text_splitter = RecursiveCharacterTextSplitter(  # 텍스트를 재귀적으로 분할하는 RecursiveCharacterTextSplitter 객체를 생성합니다.
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# HTML 헤더로 분할된 텍스트를 다시 청크 크기에 맞게 분할합니다.
splits = text_splitter.split_documents(html_header_splits)

# 분할된 텍스트 중 80번째부터 85번째까지의 청크를 출력합니다.
for header in splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

In [ ]:
for header in splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

### html tag간소화 및 AI-KMS연동

In [3]:
''' 정규표현식을 활용한 tag제거 '''
# https://dbrang.tistory.com/1149
import re
# 함수선언
def subResult(regex, repl, text):
    value = re.sub(regex, repl, text)
#     if value == text:
#         print ('# -- '+ regex, '\n', value, '\n', '-' * 30)
    return value

def delete_tag(text,method="partial"):
    if text is None:
        return ''
    pattern = r'<style[^>]*>.*?</style>|style="[^"]*"'       # style= " " 태그 제거 
    value = subResult(pattern, "", text)
    if method=='all':
        value = subResult("<(/)?p[^>]*>", "", value)        # <p~></p~> 태그 제거
        value = subResult("<(/)?b[^>]*>", "", value)        # <b~></b~> 태그 제거
        value = subResult(r">\s+<", "><", value)            # 닫는 tag와 시작tag 사이의 공백제거
    else:
        value = subResult("<p[^>]*>", "", value)              # <p~> 태그 제거
    value = subResult("<(/)?span[^>]*>", " ", value)         # <spn~></span~> 태그 제거
    value = ' '.join(value.split())                          # 두개 이상의 공백을 하나로
    value = value.replace('&nbsp;','')
    return value

In [ ]:
f_name = '52156.html'
with open(f'docs/{f_name}','r', encoding='utf-8') as f:
    data = f.readlines()
    res = ''
    for text in data:
        value= delete_tag(text)
        res += value
print(res)

o_name = f_name.split('.')[0] + 'cnv.' + f_name.split('.')[1]
with open(f'docs/{o_name}','w', encoding='utf-8') as f:
            f.write(res)

In [ ]:
import json

data = {"name":"alice"}
json_string = json.dumps(data)
print(json_string)

# with open("data.json","w") as f:
#     json.dump(data,f)

In [4]:
from pages import QA_chat as qc
from models import database as db
from models import html as htm
import json

2024-07-31 11:33:14.843 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-07-31 11:33:14.850 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [5]:
def get_sql(cn_id):
    return f'''SELECT k01.cntnt_id AS cntnt_id 
             , k01.titl AS titl_nm
             , k13.cntnt_nm AS sec_nm
             , k13.cntnt_no as sec_id
             , k24.sub_cntnt_nm AS para_nm
             , k24.sub_cntnt_no as para_id
             , k24.contn AS contn
             , bb.ctgr_path
        FROM   tbctkk01 k01
             , tbctkk13 k13
             , tbctkk24 k24
             ,(  SELECT  CTGR_ID
                        , CTGR_NM
                        , CTGR_PATH
                        , CONNECT_BY_ISLEAF AS IS_LEAF
                        FROM TBCTKK12
                        START WITH USE_YN ='Y'
                       AND CTGR_ID = 'CA'
                CONNECT BY PRIOR CTGR_ID = HGRK_CTGR_ID AND USE_YN ='Y' ) bb
        where k01.cntnt_id = {cn_id}
          AND k01.cntnt_id = k13.cntnt_id
          AND k13.cntnt_id = k24.CNTNT_ID 
          AND k13.cntnt_no = k24.CNTNT_NO 
          AND k01.use_yn = 'Y'
          AND k13.use_yn = 'Y'
          AND k24.use_yn = 'Y'
          and k01.CTGR_ID = BB.CTGR_ID
        ORDER BY k13.sort_ord, k24.sort_ord'''

In [6]:
''' 컨텐츠 생성 함수'''
def make_contents_to_json(cn_id):
    sql = get_sql(cn_id)
    df = db.get_kms_datadf(sql)
    if df is None:
        print(cn_id, ' No Data')
        return False
    df = df.fillna('')
    
    '''공공사업실 방법 ==> 최종 24.5.2 '''
    dic = {}
    temp_dic = {}
    sec_i, para_i = [0, 0]
    titl_nm,sec_id,para_id  = ["", "", ""]

    lst = []
    for idx, row in df.iterrows():
        sec_i = sec_i + 1 if row.sec_id != sec_id else sec_i
        para_i = 0 if row.sec_id != sec_id else  para_i  #텝이 바뀌면 순번 restart
        para_i = para_i + 1 if row.para_id != para_id else para_i
        temp_dic = {}
        temp_dic['title'] = row.titl_nm
        temp_dic['section'] = row.sec_nm
        temp_dic['paragraph'] = row.para_nm
#     html_string = htm.make_header(row.contn, row.titl_nm)
        html_string = row.contn
        html_string = delete_tag(html_string,"all")
        temp_dic['contents'] = html_string
        temp_dic['primary_key'] = f"{row.cntnt_id}>{row.sec_id}>{row.para_id}"
        temp_dic['path'] = row.ctgr_path
        lst.append(temp_dic)
        titl_nm, sec_id, para_id = row.titl_nm,row.sec_id,row.para_id

    o_name = 'docs/' + str(cn_id) + '.json'
    dd = dict({'data':lst})
    with open(o_name, 'w', encoding='utf8') as f:
         f.write(json.dumps(dd, ensure_ascii=False))

#     dd = str(lst)
#     with open(o_name, 'w', encoding='utf8') as f:
#          f.write(dd)
    print('succ')

In [8]:
# cn_list = [52166,52167,52168,52169] #위블비지
cn_list = [52753] #이베코,sbi
for cn_id in cn_list:
    make_contents_to_json(cn_id)

SUCCESS: Connecting database succeeded
succ


#### 생성된 컨텐츠를 API호출

In [ ]:
import requests
url = 'http://hkcloudai.com:8018/api/v1_1/vectorstore_api/vectorDB/list'

res= requests.get(url)
# data = {'param1': 'value1', 'param2': 'value'}
# res = requests.post(URL, data=data)
# res.request # 내가 보낸 request 객체에 접근 가능

# res.status_code # 응답 코드
# res.raise_for_status() # 200 OK 코드가 아닌 경우 에러 발동
res_str = res.json()# json response일 경우 딕셔너리 타입으로 바로 변환
# res.text  #UTF-8로 인코딩된 문자열
# res.content # binary

##dictonary로 key이름에 공백제거

In [ ]:
''' 1.vector list '''

url = f'http://hkcloudai.com:8018/api/v1_1/vectorstore_api/vectorDB/list'
res= requests.get(url)
res.json() ## dictionary로 리턴 필요

In [ ]:
''' 2. check contents list in vector db  '''

code = 'wible'
url = f'http://hkcloudai.com:8018/api/v1_1/vectorstore_api/vectorDB/check_contents_in_vectorDB?code={code}'
res= requests.get(url)
res.json()  ## dictionary로 리턴 필요

In [ ]:
''' 3. update contents batch'''

code = 'wiblle'
cntnt_id='52451'
url = f'http://hkcloudai.com:8018/api/v1_1/vectorstore_api/vectorDB/update_contents_batch'
f_name = f'{cntnt_id}.json'
cntnt_file = open(f'docs/{f_name}','r', encoding='utf8')
data = cntnt_file.readlines()
json_txt = {"code":code, "contents_number": cntnt_id, "files":data[0]}

api_headers = {"type" : "application/json"}
# txt_json = json.dumps(txt,ensure_ascii=False)

res = requests.put(url, data = json.dumps(json_txt,ensure_ascii=False).replace("\\",""), headers=api_headers)
# res = requests.put(url, json = txt_json, headers=api_headers)
# print(res.request.method)
# print(res.request.body)
print('res::', res.status_code)

In [ ]:
json.dumps(json_txt,ensure_ascii=False).replace("\\","")

In [ ]:
cntnt_id=52451
f_name = f'{cntnt_id}.json'
cntnt_file = open(f'docs/{f_name}','r', encoding='utf8')
data1 = cntnt_file.readlines()
data1[0]
json_txt = {"code":code, "contents_number": cntnt_id, "files":data1[0]}
json_txt


In [ ]:
jj = {
  "code": "wible",
  "contents_number": "52451",
  "files": {"data": [{"title": "이용절차 및 이용신청", "section": "이용안내", "paragraph": "이용자등록", "contents": "이용자등록 1. 교통수단을 이용하고자 하는 사람은 사전에 이용대상자 등록 절차를 거쳐야 하며, 이용대상자로 등록된 후부터 이용신청 절차를 통하여 이용할 수 있다. 2. 이용대상자 심사신청 절차 및 심사에 관한 사항은 시군 이동지원센터 에서 별도의 규정에 의거 업무를 수행한다. ※  경 기도광역이동지원시스템 (https://ggsts.gg.go.kr/front/index.do) 홈페이지 이용 1. 시ㆍ군 에 이용자 등록이 되어있는경우 &#9312; 경기도 광역이동지원시스템 회원가입(인터넷, 스마트폰앱) &#9313; 회원가입시 선택한 시ㆍ군 관할 이동지원센터로 등록요청 &#9314; 시ㆍ군 이동지원센터에서 경기도 광역이동지원시스템 가입자 아이디 확인후 승인처리 2. 시ㆍ군 에 이용자 등록이 안된경우 &#9312; 경기도 광역이동지원센터 홈페이지 회원가입(인터넷, 스마트폰) &#9313; 회원가입시 선택한 시ㆍ군 이동지원센터에 심사서류 제출 ☞ 네이버에서 OO 시ㆍ군 교통약자 이동지원센터 검색 &rarr; 이용등록신청서류 내려받기 &rarr; 서류 작성,증빙서류 제출 &#9314; 시ㆍ군 이동지원센터 승인 완료 후 이용 가능", "primary_key": "52451>1>1.1", "path": "팔레트>경기도 광역이동지원센터>표준지침>경기도광역이동"}, {"title": "이용절차 및 이용신청", "section": "이용안내", "paragraph": "이용신청", "contents": "▣특별교통수단의 이용신청은 광역 이동지원센터 전화, 모바일 앱, 홈페이지, 문자를 통해 신청 할 수 있음.  <ul ><li>이용횟수</li></ul> 즉시콜과 사전예약을 병행하며, 전체 이용횟수는 1일 4회로 제한한다. 다만, 사전예약의 이용횟수는 1일 2회로 제한한다. (차량배차 후 이용취소 건, 배차 후 미탑승 건을 모두 포함하여 산정) <ul ><li>이용시간</li></ul> 즉시콜 - 24시간 사전예약 - 이용일 기준 1일전 07시 부터 22시 사이에 선착순으로 전화, 모바일 앱, 홈페이지를 통해서 만 신청할 수 있다. <ul ><li>이용목적</li></ul> 즉시콜 - 없음 사전예약 - 병원진료, 등하교, 출퇴근시 편도로 이용 가능하며, 이를 증빙할 수 있는 서류를 사전에 해당 시군에 제출해야 한다 <table cellspacing=\"0\" cellpadding=\"0\" border=\"1\" class=\"\"><tbody><tr><th scope=\"col\"> 병원진료 </th><th scope=\"col\"> 등하교 </th><th scope=\"col\"> 출퇴근 </th></tr><tr><td > 병원 예약내역 (탑승시 운전자에게 제시)</td><td > 재학증명서</td><td > 재직증명서</td></tr></tbody></table> ※ 병원예약내역 운전자에게 미제출시 해당일부터 7일간 특별교통수단 이용이 제한된다. ※ 경기도내, 수도권 사전예약 및 즉시콜 배차 후 접수 일기준 30일이내 취소횟수가 6회 이상일 경우 특별교통수단 이용이 제한된다. <ul ><li> 즉시콜 이용자 </li></ul> 당일 접수, 당일 운행 차량 탑승을 원칙으로 한다. 단, 당일 심야차량 이용 목적인 즉시콜 접수자는 당일 심야 차량에 한해서만 탑승할 수 있다.", "primary_key": "52451>1>1.2", "path": "팔레트>경기도 광역이동지원센터>표준지침>경기도광역이동"}, {"title": "이용절차 및 이용신청", "section": "이용안내", "paragraph": "이용요금", "contents": "이용요금 1. 특별교통수단은 네비게이션에서 제공하는 실시간 최적거리 ( 유료도로 포함 ) 기준으로 운행하며 , 이를 적용한 이용요금을 부과한다 . 2 특별교통수단이용요금의 기본구간은 경기도 시내버스 요금을 적용하고 , 10km 초과시 5km 당 100 원으로 정한다 . 3 이용요금외에 차량 탑승 중 발생하는 통행료 및 주차료는 이용자 부담 을 원칙으로 한다 . ※제26조(재정지원) - 도지사는 특별교통수단에 대하여 운행요금 외에 발생하는 유료도로 통행료와 주차비는 운영비 범위내에서 지원할수있다. <table cellspacing=\"0\" cellpadding=\"0\" border=\"1\" class=\"\" ><tbody><tr><td > 시군 </td><td > 통행료부담 </td><td colspan=\"1\" rowspan=\"1\"> 주차비 </td></tr><tr><td > 가평 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 고양 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 과천 </td><td colspan=\"1\"> 이용자부담(왕복비용) </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 광명 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 광주 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 구리 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 군포 </td><td colspan=\"1\"> 센터부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr></tbody></table><table cellspacing=\"0\" cellpadding=\"0\" border=\"1\" class=\"\" ><tbody><tr><td > 김포 </td><td > 센터 부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 남양주 </td><td colspan=\"1\"> 센터 부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 동두천 </td><td colspan=\"1\"> 이용자부담(왕복비용) </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 부천 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 성남 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 수원 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 시흥 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 안산 </td><td colspan=\"1\"> 센터 부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr></tbody></table><table cellspacing=\"0\" cellpadding=\"0\" border=\"1\" class=\"\" ><tbody><tr><td > 안성 </td><td > 센터 부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 안양 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 양주 </td><td colspan=\"1\"> 센터 부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 양평 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 여주 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 연천 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 오산 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 용인 </td><td colspan=\"1\"> 센터 부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr></tbody></table><table cellspacing=\"0\" cellpadding=\"0\" border=\"1\" class=\"\" ><tbody><tr><td > 의왕 </td><td > 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 의정부 </td><td colspan=\"1\"> 센터 부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 이천 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 파주 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 평택 </td><td colspan=\"1\"> 센터 부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 포천 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 하남 </td><td colspan=\"1\"> 센터 부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr><tr><td > 화성 </td><td colspan=\"1\"> 이용자부담 </td><td colspan=\"1\" rowspan=\"1\"> 이용자부담 </td></tr></tbody></table>", "primary_key": "52451>1>1.3", "path": "팔레트>경기도 광역이동지원센터>표준지침>경기도광역이동"}]}
}
jj.keys()

In [ ]:
dict1 = { "name" : "song", "age" : 10 }
json_val = json.dumps(dict1)
json_val

In [ ]:
data[0]

In [ ]:
''' 3. update contents '''

code = 'wible'
cntnt_id='52169'
url = f'http://hkcloudai.com:8018/api/v1_1/vectorstore_api/vectorDB/update_contents?code={code}&contents_number={cntnt_id}'
f_name = f'{cntnt_id}.json'
# cntnt_file = open(f'docs/{f_name}','rb')
# files = {'file':cntnt_file}
cntnt_file = {
    'file': ('52169.json', open(f'docs/{f_name}', 'rb')),
}
headers = {"accept" : "application/json","Content-Type": "multipart/form-data"}
payload = {}
res = requests.put(url, files=files, headers=headers)
print('res::', res.status_code)

In [ ]:
cntnt_file

In [ ]:
!curl -X "PUT" \
  "http://hkcloudai.com:8018/api/v1_1/vectorstore_api/vectorDB/update_contents?code=wible&contents_number=52169" \
  -H "accept: application/json" \
  -H "Content-Type: multipart/form-data" \
  -F "files=@docs/52169.json;type=application/json"

In [ ]:
files

In [ ]:
f_name = f'{cntnt_id}.json'
with open(f'docs/{f_name}','r', encoding='utf-8') as f:
    data = f.readlines()
data[0]

In [ ]:
''' 4. retrieval '''

code = 'CW'
query = '보상쿠폰 지급기준에 대해 알려줘'
url = f'http://hkcloudai.com:8018/api/v1_1/retrieve/retrieve/retrieval_non_streaming?code={code}&query={query}'
res = requests.get(url)
res_str = res.json()  ## dictionary로 넣어줄 필요가 있음
res_str['llm_response'][0]


In [ ]:
# res_str['llm_response']
# res_str['retrieval_docs']
for i in res_str.get('retrieval_docs'): #'meta_data'
    print(i.get('doc'))
    cntnt_id = i.get('metadata').get('primary_key').split('>')[0]
    section =i.get('metadata').get('section')
    para = i.get('metadata').get('paragraph')
    rel_doc = f'{cntnt_id} : {section} >  {para}'
    print(rel_doc)
    
    k_url = 'https://nkms.hkpalette.com/webapps/kk/kn/KkKn003.jsp?TYPE=KN&CNTNT_ID='
    k_url = f'{k_url}{cntnt_id}&MENU_ID='
    print(k_url)
    print(f'[{rel_doc}]({k_url}')

#### 아래는 참고용

In [ ]:
# cn_id = 52162
# # df = qc.get_kms_contents(cn_id)
# sql = get_sql(cn_id)
# df = db.get_kms_datadf(sql)

# '''간단한 방법이며 이 방법으로 일단 진행시킴'''
# dic = {}
# temp_dic = {}
# titl_nm,tab_nm,sub_nm  = ['', '', '']
# sec_i, para_i = [0, 0]

# for idx, row in df.iterrows():
#     sec_i = sec_i + 1 if row.tab_nm != tab_nm else sec_i
#     para_i = para_i + 1 if row.sub_nm != sub_nm else para_i

#     temp_dic['titl'] = row['titl_nm']
#     temp_dic['section'+str(sec_i)] = row.tab_nm
#     temp_dic['paragraph'+str(para_i)] = row.sub_nm
#     html_string = htm.make_header(row.contn, row.titl_nm)
#     html_string = delete_tag(html_string)
#     temp_dic['contents'+str(para_i)] = html_string
#     dic.update(temp_dic)
#     titl_nm, tab_nm, sub_nm = row.titl_nm,row.tab_nm,row.sub_nm
    
# metadata = f'id={df.cntnt_id[0]};path={df.ctgr_path[0]}'
# dd = {'data':dic, 'metadata': metadata}
# o_name = 'docs/' + str(cn_id) + '.json'
# with open(o_name, 'w', encoding='utf8') as f:
#      f.write(json.dumps(dd, ensure_ascii=False))
# print('succ')

In [ ]:
cn_id = 52156
# df = qc.get_kms_contents(cn_id)
sql = get_sql(cn_id)
df = db.get_kms_datadf(sql)

'''공공사업실 방법 ==> 협의했던 방법(순번부여)'''
dic = {}
temp_dic = {}
titl_nm,sec_nm,para_nm  = ['', '', '']
sec_i, para_i = [0, 0]

lst = []
for idx, row in df.iterrows():
    sec_i = sec_i + 1 if row.sec_nm != sec_nm else sec_i
    para_i = 0 if row.sec_nm != sec_nm else  para_i  #텝이 바뀌면 순번 restart
    para_i = para_i + 1 if row.para_nm != para_nm else para_i
    temp_dic = {}
    temp_dic['section'+str(sec_i)] = row.sec_nm
    temp_dic['paragraph'+str(para_i)] = row.para_nm
#     html_string = htm.make_header(row.contn, row.titl_nm)
    html_string = row.contn
    html_string = delete_tag(html_string,"all")
    temp_dic['contents'+str(para_i)] = html_string
    lst.append(temp_dic)
    titl_nm, sec_nm, para_nm = row.titl_nm,row.sec_nm,row.para_nm
    
# metadata = f'title:{titl_nm},primary_key:{df.cntnt_id[0]},path={df.ctgr_path[0]}'
metadata = dict({'title':titl_nm,'primary_key':df.cntnt_id[0], 'path':df.ctgr_path[0]})
dd = dict({'data':lst, 'metadata':metadata})
o_name = 'docs/' + str(cn_id) + '.json'
with open(o_name, 'w', encoding='utf8') as f:
     f.write(json.dumps(dd, ensure_ascii=False))
print('succ')

In [ ]:
### 트리 구조형태이나 너무 복잡함 ###
dic = {}
titl_nm,sec_nm,para_nm  = ['', '', '']
sec_i, para_i = [0, 0]

for idx, row in df.iterrows():
#     print(row)
    sec_i = sec_i + 1 if row.sec_nm != sec_nm else sec_i
    print(sec_i, row.sec_nm)
    para_i = para_i + 1 if row.para_nm != para_nm else para_i

    dic['titl'] = row['titl_nm']
    dic['contents '+str(sec_i)] = {'title': row.sec_nm,
                      "para": {
                        'para'+str(para_i):
                            { 'title': row.para_nm,
                              'contents': "ss" #row.contn
                          }
                       }
                     }
    titl_nm, sec_nm, para_nm = row.titl_nm,row.sec_nm,row.para_nm

In [ ]:
df.groupby(level=-1).apply(lambda df: df.xs(df.name).to_dict()).to_dict()
# df.groupby(level=0)

In [ ]:
df.xs(df.name)

In [ ]:
for i in df.groupby(level=0):
    print(i)

In [ ]:
data_json4 = {
     "title": "제목명",
     "section": {
        "sec1" : {
            "title":"탭1명",
            "para": {
              "para1": { 
                "title" : "소제목1명",
                "contents": "내용1 html"
              },
              "para2": { 
                "title" : "소제목1명",
                "contents": "내용2 html"
             }
            }
          },

          "sec2" : {
            "title":"탭2명",
            "para": {
                "para1": { 
                    "title" : "소제목1명",
                    "contents": "내용1 html"
                },
                "para2": { 
                    "title" : "소제목1명",
                    "contents": "내용1 html"
                },
            }
          }
        }
      }